# Individual Assignment #3
Matteo Bordoni - 119474303

 The goal of this project is to help you learn data merge, group comparison, and basic regression analysis.

 

The project will use two datasets:

The “long sample” you have created in Project 1, which should already have each observation defined as a place-day, and include variables such as date, dailyvisits, dayofweek, and weekend in each observation. By definition, the data focuses on your state.
A cross-sectional data from Safegraph called “core place records.” Every observation in this data is a business place, with a unique Safegraph id and its industry code (6-digit NAICS), brand affiliation, etc. This nationwide data is available on ELMS ==> week 8 module.
When you turn in your jupyter notebook on ELMS (in both .ipynb and .pdf), please label clearly your name and the State you are focusing on. For example, if your name is Joe Smith and you focus on Maryland, you should name your project 3 submissions as Econ433-proj3-Joe-Smith-MD.ipynb and Econ433-proj3-Joe-Smith-MD.pdf.

Within your jupyter notebook, please use markdown cells to describe which python commands and outcome are for which question in the project. Make sure your notebook is reader-friendly, includes all key information, and is free of unnecessary text (such as the error message before you fix the problem).

 

### Part 1: Prepare data from core place records

 

- Question 1a (2 point): Download the nationwide “core place records” from ELMS ==> week 8. In your python notebook, read in this data, and save the records of your state into a separate data file. For reference, I will call this data “StateCorePlaces” here. You can use a different name in your project.

 

How many observations do you have in “StateCorePlaces”? How many unique NAICS codes are there in the data? According to the Census's definition of NAICS (here), Links to an external site.what industries do these NAICS codes represent? How many observations are there for each unique NAICS code?

 

- Question 1b (2 point): In the “StateCorePlaces” data you have created in Question 1a, define a variable “bizgroup” equal to 1 if a place’s NAICS code falls in a range you are really interested in, and 0 if a place's NAICS code falls in a range you want to use as a comparison group. Places that do not fall in either range will have a missing value in the bizgroup, or they should be dropped from your data.

For example, NAICS code 445110 stands for “Supermarkets and Other Grocery (except Convenience) Stores”, 445120 stands for “Convenience Stores”, 722511 stands for "Full Service Restaurants", 722513 stands for “Limited-Service Restaurants”, 611110 stands for "primary and secondary schools", and 611310 for "Colleges, universities, and professional schools". If you want to contrast full service restaurants with every other types of restaurants, you can define bizgroup=1 for any places that have a NAICS code 722511, and bizgroup=0 if the place has a NAICS code 722513, 722514, or 722515. Every place else will either have a missing value in bizgroup, or have been dropped from your data.

 

Feel free to choose other definitions as you like, each group can contain one or more NAICS codes. Whatever you choose, state clearly what each group represents, why you are interested in this comparison, and how you expect DailyVisits to differ between these two groups.

 

In the core places data, how many places have bizgroup=0? how many places have bizgroup=1?

 

Find the top ten cities in your state that have the highest count of unique places (across all NAICS codes). For these ten cities, count # of places by bizgroup=0 and bizgroup=1 in every city. You can use a table to present this result, where each row is a city, one column shows the count of places with bizgroup=0, and another column shows the count of places with bizgroup=1. 

 

If your state is very large and thus you have focused on one or few cities of the state in Project 1, you can complete this task by top ten zipcodes rather than top ten cities.

 

### Part 2: Merge daily visits data with StateCorePlaces

 

- Question 2a (2 point): Merge StateCorePlaces with the “long” sample you have used in Project 1, by Safegraph place id (placekey).

 

How many observations are matched? How many observations appear in StateCorePlaces but not in the “long” sample? How many observations appear in the “long” sample but not in “StateCorePlaces”?

 

- Question 2b (1 point): Keep the matched records from the merge in Question 2a; keep only the places that have either bizgroup=0 or bizgroup=1 in your top ten cities. Show the number of observations for each unique combination of bizgroup and naics_code (hint: groupby). Does this agree with how you defined bizgroup in 1b?

 

### Part 3: Group comparison in the merged data

 

- Question 3a (2 points): Focus on the “final” data. Calculate average dailyvisits for bizgroup=0 and bizgroup=1.  Show the results in a table.  How does the average dailyvisits differ between the two groups? (Write at least one sentence about your takeaway from the table.)

 

- Question 3b (2 points): Perform a statistical test on the difference of average dailyvisits between bizgroup=0 and bizgroup=1. Is the difference statistically significant from zero with 95% confidence? Does it confirm your prior in Question 3a?

 

### Part 4: Basic regression analysis

 

- Question 4a (2 points): Focus on the “final” data. Define a dummy “Post0313” equal to one if date is at or after March 13, 2020. Define “bizgroup_post” as bizgroup times Post0313. Regress ln(dailyvisits+1) on bizgroup, Post0313, bizgroup_post, and weekend.

 

Comment on the estimated coefficient for each of the four variables: Are they statistically significant from zero with 95% confidence? What is the economic meaning of each coefficient? Do you accept or reject the null hypothesis “H0: The national emergency announced on March 13 does not make a different on average dailyvisits to the two groups of businesses”?

 

- Question 4b (2 points): How would your answer to Question 4a change if (1) you add city fixed effects? (2) you add date fixed effects? and (3) you add both city fixed effects and date fixed effects? Which specification would you prefer for the hypothesis testing?



## Setting Up the Work Space 

Before starting to work, it is necessary to set up the working space, importing all the libraries necessary to complete the task 

In [1]:
import pandas as pd 
import numpy as np
from scipy.stats import ttest_ind
import statsmodels.formula.api as smf
from linearmodels import PanelOLS

import warnings
warnings.filterwarnings("ignore")

## Importing the Data 

The work is started by importing and loading the data into the working space. The work will consider two different dataframes: a long sample, which has been created in the past assignments, and a new dataframe with state core places. 

In [2]:
name = 'long_sample.csv'
long_sample = pd.read_csv(name)
long_sample.head()

,placekey,city,region,date,daily_visits,day_of_week,positive_visits,weekend
0,zzw-224@5px-7c2-z75,Louisville,KY,2018-02-05,1,Monday,1,0
1,zzw-223@63m-v37-tqf,Ashland,KY,2018-02-05,10,Monday,1,0
2,222-222@5pn-ffm-y5f,Madisonville,KY,2018-02-05,3,Monday,1,0
3,222-222@8g9-rgb-26k,Burnside,KY,2018-02-05,0,Monday,0,0
4,222-223@5px-8zj-7yv,Fairdale,KY,2018-02-05,6,Monday,1,0


In [3]:
name_2 = 'core_places_2018-2022_sample.csv'
state_core_places = pd.read_csv(name_2)
state_core_places.head()

,placekey,parent_placekey,safegraph_brand_ids,naics_code,postal_code,region,city,distance_from_home,median_dwell,poi_cbg
0,222-222@3bc-tnz-ffz,NaN,NaN,721110,99631.0,AK,Moose Pass,NaN,28.392857,21220003001
1,222-222@3bc-yjz-vpv,NaN,NaN,722511,99587.0,AK,Girdwood,NaN,88.000000,20200029002
2,222-222@3bg-qd7-qpv,NaN,NaN,611110,99639.0,AK,Ninilchik,NaN,19.714286,21220008001
3,222-222@3bh-25s-zmk,NaN,NaN,611110,99603.0,AK,Fritz Creek,3414.000000,8.901235,21220011002
4,222-222@3bh-2b5-j5f,NaN,NaN,721110,99603.0,AK,Homer,47769.761905,88.095238,21220010001


In [4]:
KY_core = state_core_places[state_core_places['region'] == 'KY']
KY_core.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3590 entries, 1560 to 276356
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   placekey             3590 non-null   object 
 1   parent_placekey      608 non-null    object 
 2   safegraph_brand_ids  1165 non-null   object 
 3   naics_code           3590 non-null   int64  
 4   postal_code          3590 non-null   float64
 5   region               3590 non-null   object 
 6   city                 3590 non-null   object 
 7   distance_from_home   2532 non-null   float64
 8   median_dwell         3590 non-null   float64
 9   poi_cbg              3590 non-null   int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 308.5+ KB


In [5]:
KY_core.describe()

,naics_code,postal_code,distance_from_home,median_dwell,poi_cbg
count,3590.000000,3590.000000,2532.000000,3590.000000,3.590000e+03
mean,653894.075766,41088.153482,26577.081932,78.756505,2.111071e+11
std,101121.911582,887.419487,67453.713273,148.899785,6.186547e+08
min,445110.000000,40004.000000,500.293447,4.000000,2.100197e+11
25%,611110.000000,40291.000000,6191.571697,13.504041,2.106700e+11
50%,722511.000000,40831.000000,9252.200288,35.000000,2.111101e+11
75%,722511.000000,42064.000000,13841.647613,76.000172,2.115101e+11
max,722513.000000,42784.000000,762034.945990,1413.000000,2.123905e+11


In [6]:
KY_core.head()

,placekey,parent_placekey,safegraph_brand_ids,naics_code,postal_code,region,city,distance_from_home,median_dwell,poi_cbg
1560,222-222@5pj-89c-pn5,NaN,SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,42029.0,KY,Calvert City,34070.890811,41.274595,211579501004
1561,222-222@5pj-8bx-v9f,NaN,SG_BRAND_efc6bd8205d5d857fcbb3d2be569d983,721110,42045.0,KY,Grand Rivers,462400.983607,127.245902,211390402005
1565,222-222@5pj-8dk-xt9,NaN,SG_BRAND_1f852a23da4b7250,445110,42001.0,KY,Paducah,6653.017908,17.992460,211450314001
1566,222-222@5pj-8dn-d9z,NaN,NaN,611110,42003.0,KY,Paducah,6158.400359,45.755835,211450307004
1567,222-222@5pj-8dp-575,NaN,SG_BRAND_2b6b8e67fe1433e4f73bbe2c306fb330,722513,42001.0,KY,Paducah,7290.376623,24.878788,211450305001


### Analyzing the Data

In [7]:
naics_table = pd.DataFrame(KY_core.naics_code.value_counts())
naics_table = naics_table.rename(columns={'naics_code': 'count'}, index=str).rename_axis('naics_code')
naics_table 

,count
naics_code,
722511,1113
722513,823
445110,405
624410,387
611110,363
721110,264
445120,175
611310,60


From the Census website, it is matched the corresponding code to its industry

In [8]:
def map_industry(code):
    if code == '722511':
        return 'Full-service restaurants'
    elif code == '722513':
        return 'Limited-service restaurants'
    elif code == '445110':
        return 'Supermarkets and other grocery stores'
    elif code == '624410':
        return 'Child day care services'
    elif code == '611110':
        return 'Elementary and secondary schools'
    elif code == '721110':
        return 'Hotels (except casino hotels) and motels'
    elif code == '445120':
        return 'Convenience stores'
    elif code == '611310':
        return 'Colleges, universities, and professional schools'
    else:
        return 'Unknown'

In [9]:
naics_table['industry'] = naics_table.apply(lambda x: map_industry(x.name), axis=1)
naics_table 

,count,industry
naics_code,,
722511,1113,Full-service restaurants
722513,823,Limited-service restaurants
445110,405,Supermarkets and other grocery stores
624410,387,Child day care services
611110,363,Elementary and secondary schools
721110,264,Hotels (except casino hotels) and motels
445120,175,Convenience stores
611310,60,"Colleges, universities, and professional schools"


### Focusing on an industry

The designated industry is the restaurant market. The following analysis will focus on the differences between full-service restaurants (722511) and limited-service restaurants (722513). The choice was operated as these two categories contains the most number of observations and an analysis could be interesting to understand differences between the two markets. 

In [10]:
KY_core['bizgroup'] = KY_core['naics_code'].apply(lambda x: 1 if x == 722511 else (0 if x == 722513 else np.nan))
KY_core['bizgroup'] = KY_core['bizgroup'].astype('Int64')
KY_core.head()

,placekey,parent_placekey,safegraph_brand_ids,naics_code,postal_code,region,city,distance_from_home,median_dwell,poi_cbg,bizgroup
1560,222-222@5pj-89c-pn5,NaN,SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,42029.0,KY,Calvert City,34070.890811,41.274595,211579501004,1
1561,222-222@5pj-8bx-v9f,NaN,SG_BRAND_efc6bd8205d5d857fcbb3d2be569d983,721110,42045.0,KY,Grand Rivers,462400.983607,127.245902,211390402005,<NA>
1565,222-222@5pj-8dk-xt9,NaN,SG_BRAND_1f852a23da4b7250,445110,42001.0,KY,Paducah,6653.017908,17.992460,211450314001,<NA>
1566,222-222@5pj-8dn-d9z,NaN,NaN,611110,42003.0,KY,Paducah,6158.400359,45.755835,211450307004,<NA>
1567,222-222@5pj-8dp-575,NaN,SG_BRAND_2b6b8e67fe1433e4f73bbe2c306fb330,722513,42001.0,KY,Paducah,7290.376623,24.878788,211450305001,0


In [11]:
KY_core = KY_core.dropna(subset=['bizgroup'])
KY_core.head()

,placekey,parent_placekey,safegraph_brand_ids,naics_code,postal_code,region,city,distance_from_home,median_dwell,poi_cbg,bizgroup
1560,222-222@5pj-89c-pn5,NaN,SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511,42029.0,KY,Calvert City,34070.890811,41.274595,211579501004,1
1567,222-222@5pj-8dp-575,NaN,SG_BRAND_2b6b8e67fe1433e4f73bbe2c306fb330,722513,42001.0,KY,Paducah,7290.376623,24.878788,211450305001,0
1572,222-222@5pj-8fh-n89,NaN,SG_BRAND_8125123123f1ed7b7bda7e85e570dd52,722511,42001.0,KY,Paducah,28578.081272,28.632509,211450314001,1
1575,222-222@5pj-8m9-wc5,NaN,SG_BRAND_75e52dcc790fbad91ae83227c0fb6e2f,722513,42003.0,KY,Paducah,11750.006711,9.416107,211450301001,0
2301,222-222@5pn-cb5-rrk,NaN,NaN,722511,42071.0,KY,Murray,2244.769811,60.537736,210350103013,1


In [12]:
num_obs = pd.DataFrame(KY_core.bizgroup.value_counts())
num_obs = num_obs.rename(columns={'bizgroup': 'count'}).rename_axis('bizgroup')
num_obs

,count
bizgroup,
1,1113
0,823


Then, a dataset with the ten cities with the most observations is created 

In [13]:
places_by_city = KY_core.groupby('city')['placekey'].nunique()
top_cities = places_by_city.nlargest(10).index
counts = {}
for city in top_cities:
    city_data = KY_core[KY_core['city'] == city]
    bizgroup_counts = city_data['bizgroup'].value_counts(dropna=False)
    counts[city] = {
        'group_0': bizgroup_counts.get(0, 0),
        'group_1': bizgroup_counts.get(1, 0)
    }
cities_table = pd.DataFrame.from_dict(counts, orient='index')
cities_table

,group_0,group_1
Louisville,138,224
Lexington,69,124
Bowling Green,23,41
Florence,15,37
Owensboro,13,21
Covington,7,25
Elizabethtown,10,22
Frankfort,13,18
Paducah,15,15
Richmond,13,17


### Preparing the Dataset

In this section, we will focus on merging the two dataset to get to a final stage in which we can use the pandas dataframe to operate some regressions on it.

In [14]:
merged_data = KY_core.merge(long_sample, on='placekey', how='outer')
merged_data.head()

,placekey,parent_placekey,safegraph_brand_ids,naics_code,postal_code,region_x,city_x,distance_from_home,median_dwell,poi_cbg,bizgroup,city_y,region_y,date,daily_visits,day_of_week,positive_visits,weekend
0,222-222@5pj-89c-pn5,NaN,SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511.0,42029.0,KY,Calvert City,34070.890811,41.274595,2.115795e+11,1,Calvert City,KY,2018-02-19,14,Monday,1,0
1,222-222@5pj-89c-pn5,NaN,SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511.0,42029.0,KY,Calvert City,34070.890811,41.274595,2.115795e+11,1,Calvert City,KY,2018-02-05,20,Monday,1,0
2,222-222@5pj-89c-pn5,NaN,SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511.0,42029.0,KY,Calvert City,34070.890811,41.274595,2.115795e+11,1,Calvert City,KY,2018-02-12,20,Monday,1,0
3,222-222@5pj-89c-pn5,NaN,SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511.0,42029.0,KY,Calvert City,34070.890811,41.274595,2.115795e+11,1,Calvert City,KY,2018-02-26,15,Monday,1,0
4,222-222@5pj-89c-pn5,NaN,SG_BRAND_4c72c81de7cea539cb09bb1344c7b367,722511.0,42029.0,KY,Calvert City,34070.890811,41.274595,2.115795e+11,1,Calvert City,KY,2018-07-30,27,Monday,1,0


In [15]:
num_matched = merged_data.shape[0] - merged_data['placekey'].isna().sum()
num_in_KY_core = (merged_data['city_x'].notna() & merged_data['city_y'].isna()).sum()
num_in_long_sample = (merged_data['city_x'].isna() & merged_data['city_y'].notna()).sum()

print(f"Number of matched observations: {num_matched}")
print(f"Number of observations in KY_core but not in long_sample: {num_in_KY_core}")
print(f"Number of observations in long_sample but not in KY_core: {num_in_long_sample}")

Number of matched observations: 4771452
Number of observations in KY_core but not in long_sample: 0
Number of observations in long_sample but not in KY_core: 2261462


In [16]:
top_ten_cities = KY_core.groupby('city')['placekey'].nunique().nlargest(10).index.tolist()
KY_core_filtered = KY_core[(KY_core['city'].isin(top_ten_cities)) & (KY_core['bizgroup'].isin([0,1]))]
final_data = pd.merge(KY_core_filtered, long_sample, on=['placekey', 'city', 'region'], how='inner')
final_data.head()

,placekey,parent_placekey,safegraph_brand_ids,naics_code,postal_code,region,city,distance_from_home,median_dwell,poi_cbg,bizgroup,date,daily_visits,day_of_week,positive_visits,weekend
0,222-222@5pj-8dp-575,NaN,SG_BRAND_2b6b8e67fe1433e4f73bbe2c306fb330,722513,42001.0,KY,Paducah,7290.376623,24.878788,211450305001,0,2018-02-19,6,Monday,1,0
1,222-222@5pj-8dp-575,NaN,SG_BRAND_2b6b8e67fe1433e4f73bbe2c306fb330,722513,42001.0,KY,Paducah,7290.376623,24.878788,211450305001,0,2018-02-26,6,Monday,1,0
2,222-222@5pj-8dp-575,NaN,SG_BRAND_2b6b8e67fe1433e4f73bbe2c306fb330,722513,42001.0,KY,Paducah,7290.376623,24.878788,211450305001,0,2018-02-05,3,Monday,1,0
3,222-222@5pj-8dp-575,NaN,SG_BRAND_2b6b8e67fe1433e4f73bbe2c306fb330,722513,42001.0,KY,Paducah,7290.376623,24.878788,211450305001,0,2018-02-12,3,Monday,1,0
4,222-222@5pj-8dp-575,NaN,SG_BRAND_2b6b8e67fe1433e4f73bbe2c306fb330,722513,42001.0,KY,Paducah,7290.376623,24.878788,211450305001,0,2018-07-16,8,Monday,1,0


In [17]:
final_data['date'] = pd.to_datetime(final_data['date'])

### Data Comparison

In [18]:
avg_daily_visits = pd.DataFrame(final_data.groupby('bizgroup')['daily_visits'].mean())
avg_daily_visits

,daily_visits
bizgroup,
0,15.937144
1,11.882020


As expected, the group 1, which is full-service restaurants, have a lower daily visits average; it is reasonable to believe that limited service restaurants, such as fast food, have an higher daily visits average 

In [19]:
group0_visits = final_data.loc[final_data['bizgroup'] == 0, 'daily_visits']
group1_visits = final_data.loc[final_data['bizgroup'] == 1, 'daily_visits']
t_statistic, p_value = ttest_ind(group0_visits, group1_visits)
print(f"t-statistic: {t_statistic:.2f}")
print(f"p-value: {p_value:.3f}")
alpha = 0.05 
if p_value < alpha:
    print("The difference in average daily visits between bizgroup=0 and bizgroup=1 is statistically significant at the 95% confidence level.")
else:
    print("The difference in average daily visits between bizgroup=0 and bizgroup=1 is not statistically significant at the 95% confidence level.")

t-statistic: 82.56
p-value: 0.000
The difference in average daily visits between bizgroup=0 and bizgroup=1 is statistically significant at the 95% confidence level.


Given that the p-value for the variable "bizgroup" is 0.000, which is less than the significance level of 0.05, the coefficient for this variable is statistically significant from zero with 95% confidence. This enables us to infer that there is a statistically significant difference in the average daily visits between the two groups of firms and reject the null hypothesis that the coefficient for "bizgroup" is zero. This is a confirm of what it was believed above. 

### Regression Analysis 

This section focuses on the actual regression analysis. First of all, a dummy variable 'Post0313' is created to have a better understanding of the covid-19 effect on the dailyvists. Then, a variable 'bizgroup' with the product of 'bizgroup' and 'Post0313' is created. Then, we are all set to compute the first Ordinary Least Square Regression.

In [20]:
final_data['Post0313'] = (final_data['date'] >= '2020-03-13').astype(int)
final_data['bizgroup_post'] = final_data['bizgroup'] * final_data['Post0313']

In [21]:
final_data['bizgroup'] = final_data['bizgroup'].astype(float)
final_data['bizgroup_post'] = final_data['bizgroup_post'].astype(float)
model = smf.ols(formula='np.log(daily_visits+1) ~ bizgroup + Post0313 + bizgroup_post + weekend', data=final_data)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     np.log(daily_visits + 1)   R-squared:                       0.021
Model:                                  OLS   Adj. R-squared:                  0.021
Method:                       Least Squares   F-statistic:                     6034.
Date:                      Fri, 31 Mar 2023   Prob (F-statistic):               0.00
Time:                              16:57:38   Log-Likelihood:            -1.7728e+06
No. Observations:                   1148917   AIC:                         3.546e+06
Df Residuals:                       1148912   BIC:                         3.546e+06
Df Model:                                 4                                         
Covariance Type:                  nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         2.1986      0.003    858.199      0.000       2.194       2.204
bizgroup         -0.3169      0.003   -100.035      0.000      -0.323      -0.311
Post0313          0.0749      0.003     22.561      0.000       0.068       0.081
bizgroup_post     0.0024      0.004      0.549      0.583      -0.006       0.011
weekend          -0.0668      0.002    -28.580      0.000      -0.071      -0.062
==============================================================================
Omnibus:                    10634.850   Durbin-Watson:                   0.379
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6908.601
Skew:                           0.021   Prob(JB):                         0.00
Kurtosis:                       2.622   Cond. No.                         8.02
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The intercept coefficient represents the estimated average log(daily_visits + 1) for the reference group when all other variables in the model are equal to zero. All the intercept in the model are statistically significant except for 'bizgroup_post' which has a p-value greater than 0.05. The coeffient could be interpresent as the effect of each variable on the outcome variable. This means that whenever the coeffient is positive, the variable will have a positive effect on 'daily_visits' and negative otherwise. However, the R-squared, which is a statistical measure that represents the proportion of the variance in the dependent variable (y) that is explained by the independent variables (x) in a linear regression model, is very low, meaning that the measure of how well the regression model fits the observed data is not accurate. In fact, in this Ordinary Least Squared regression results, the R-squared value is 0.021, which indicates that the independent variables (bizgroup, Post0313, bizgroup_post, and weekend) in the model explain only 2.1% of the variation in the dependent variable (np.log(daily_visits + 1)). We may look at the coefficient for "bizgroup post" to test the null hypothesis "H0: The national emergency declared on March 13 does not affect the average daily visits to the two groups of companies." We fail to reject the null hypothesis since the p-value for this coefficient is 0.583, which is higher than 0.05. Thus, there is no statistically significant difference between the impact of the national emergency on daily visits to non-food businesses and businesses that sell food.

#### Fixed Effect Regression

To study the fixed effect, it is needed to perform a panel data regression, having the data distributed across cities and across dates.

In [22]:
final_data_reg = final_data.copy()
final_data_reg = final_data_reg.set_index(['city', 'date'])

In [23]:
def log_plus_1(x):
    return np.log(x + 1)

In [24]:
mod = PanelOLS(log_plus_1(final_data_reg['daily_visits']), 
               final_data_reg[['bizgroup', 'Post0313', 'bizgroup_post', 'weekend']], 
               entity_effects=True,
               drop_absorbed = True)
results = mod.fit(cov_type='clustered')
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:           daily_visits   R-squared:                        0.0224
Estimator:                   PanelOLS   R-squared (Between):             -0.1742
No. Observations:             1148917   R-squared (Within):               0.0224
Date:                Fri, Mar 31 2023   R-squared (Overall):             -0.1305
Time:                        16:57:42   Log-likelihood                -1.756e+06
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6588.6
Entities:                          10   P-value                           0.0000
Avg Obs:                    1.149e+05   Distribution:               F(4,1148903)
Min Obs:                    3.799e+04                                           
Max Obs:                    5.034e+05   F-statistic (robust):             6571.7
                            

In this instance, the independent variables in the model only partially explain the variation in daily visits, as shown by the R-squared value of 0.0224. Together with their standard errors, t-statistics, and p-values, the estimated coefficients for each independent variable are presented. Indicating that these variables significantly affect daily visits, the coefficients for bizgroup and Post0313 are statistically significant at the 95% confidence level. The weekend coefficient is statistically significant as well, although it has a negative sign, indicating that companies can see fewer daily visitors on weekends. Since the bizgroup post coefficient is not statistically significant, there is no discernible difference between the two business groups in the impact of Post0313. Lastly, the model's poolability F-test and corresponding p-value show whether significant fixed effects for entities are present in the model. The inclusion of entity fixed effects improves the model's fit, as the p-value of 0.0000 suggests that there is sufficient evidence to reject the null hypothesis that the fixed effects for entities are not significant.

In [25]:
mod = PanelOLS(log_plus_1(final_data_reg['daily_visits']), 
               final_data_reg[['bizgroup', 'Post0313', 'bizgroup_post', 'weekend']], 
               entity_effects=False,
               time_effects=True,
               drop_absorbed=True)
results = mod.fit(cov_type='clustered')
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:           daily_visits   R-squared:                        0.0192
Estimator:                   PanelOLS   R-squared (Between):             -0.1855
No. Observations:             1148917   R-squared (Within):               0.0205
Date:                Fri, Mar 31 2023   R-squared (Overall):             -0.1404
Time:                        16:57:51   Log-likelihood                -1.745e+06
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   1.124e+04
Entities:                          10   P-value                           0.0000
Avg Obs:                    1.149e+05   Distribution:               F(2,1147088)
Min Obs:                    3.799e+04                                           
Max Obs:                    5.034e+05   F-statistic (robust):          1.138e+04
                            

In this instance, the R-squared value is 0.0192, indicating that only 1.92% of the variation in daily visits can be accounted for by the independent variables in the model. The F-statistic evaluates the model's overall significance. The F-statistic in this instance is 1.124e+04, which is extremely high and shows that the model is statistically significant. Overall, the model shows a statistically significant negative connection between daily visits and the independent variable bizgroup. The independent variables in the model do not adequately account for most of the variation in the dependent variable, as seen by the low R-squared value. There is no correlation between bizgroup post and daily visits, as shown by the fact that the parameter estimate for bizgroup post is not statistically significant. The statistical significance of the F-test for poolability shows that there is a substantial difference between the model's entities.

In [26]:
mod = PanelOLS(log_plus_1(final_data_reg['daily_visits']), 
               final_data_reg[['bizgroup', 'Post0313', 'bizgroup_post', 'weekend']], 
               entity_effects=True,
               time_effects=True,
               drop_absorbed=True)
results = mod.fit(cov_type='clustered')
print(results)

                          PanelOLS Estimation Summary                           
Dep. Variable:           daily_visits   R-squared:                        0.0211
Estimator:                   PanelOLS   R-squared (Between):             -0.1930
No. Observations:             1148917   R-squared (Within):               0.0206
Date:                Fri, Mar 31 2023   R-squared (Overall):             -0.1462
Time:                        16:58:04   Log-likelihood                -1.727e+06
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   1.237e+04
Entities:                          10   P-value                           0.0000
Avg Obs:                    1.149e+05   Distribution:               F(2,1147079)
Min Obs:                    3.799e+04                                           
Max Obs:                    5.034e+05   F-statistic (robust):          1.262e+04
                            

The very low R-squared value (0.0211) indicates that the model does not adequately account for the variation in daily visits. The model's F-statistic evaluates the importance of the independent variables as a whole. Given that the F-statistic is quite high (1.237e+04), at least one independent variable must be significant. Overall, the regression's findings indicate that bizgroup, but not bizgroup post, has a substantial impact on daily visits. The R-squared value, however, is quite low, indicating that there are more variables that influence daily visits but are not accounted for by the model.